In [ ]:
# ============= Modules
# Install PYCBC
!pip install -q pycbc
import pycbc

#Import pycbc functions
import pycbc.waveform
import pycbc.noise
import pycbc.psd
import pycbc.filter
import pycbc.conversions

# Import Drive
from google.colab import drive

# Import DataVisualization libraries
from matplotlib import pyplot as plt

#Import Algebra tools
import numpy as np
import pandas as pd
import scipy as sc
from scipy import interpolate
import scipy.signal  as sg
from tqdm import tqdm
import random as ran

import copy
import glob

In [ ]:
def resample(t,h,fs):
    """
    Interpolate array h to the fs sampling frequency.
    Input:
        t  - time array, in seconds
        h  - strain array to be interpolated
        fs - thi is the new sampling frequency
    Output:
        t1 - time array, after resampling
        h1 - new strain array
    """
    # Quick check
    if len(t)!=len(h):
        print("Error: t and h need to have equal sizes")
        return 0
    # Define new time with fs
    t1 = np.arange(t[0],t[-1],1.0/fs)
    # Interpolation
    tck = interpolate.splrep(t,h,s=0)
    h1  = interpolate.splev(t1,tck,der=0)

    return t1, h1

In [ ]:
# Mount Personal Drive
drive.mount('/content/drive')
L1_asd = pd.read_csv("/content/drive/MyDrive/Thesis/PYCBC/O3-L1-C01-1262141640.0_sensitivity_strain_asd.txt",header=None,names=["Freq","ASD"],sep="\t")
H1_asd = pd.read_csv("/content/drive/MyDrive/Thesis/PYCBC/O3-H1-C01-1262197260.0_sensitivity_strain_asd.txt",header=None,names=["Freq","ASD"],sep="\t")
V1_asd = pd.read_csv("/content/drive/MyDrive/Thesis/PYCBC/O3-H1-C01-1262197260.0_sensitivity_strain_asd.txt",header=None,names=["Freq","ASD"],sep="\t")
#V1_asd = pd.read_csv("")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Phenomenological Direction in drive
files = sorted(glob.glob("/content/drive/MyDrive/Thesis/phen_data/*.csv"))
# i=0
# for j in range(0,len(files)):
#     print(f"No.{j}: {files[j]}")
#     i+=1

s = pd.read_csv(files[1])
url1 = "/content/drive/MyDrive/Thesis/Slow_run2H13_Slope3144_f0(124.22)_f1(1903.02).csv"
url2 = "/content/drive/MyDrive/Thesis/Slow_run2H198_Slope3168_f0(123.62)_f1(1882.91).csv"
# Dataframe of the Phen data
df = pd.read_csv(url1)
df = pd.read_csv(url2)

# ======== Read Phenomenological WF

phen_t = df["Time"]
phen_amp = df["H"]


phen_t   = np.array(phen_t)
phen_amp = np.array(phen_amp)

# Code dissert that is in 4096 hz
phen_t,phen_amp = resample(phen_t,phen_amp,4096)
# print(len(phen_amp))

In [ ]:
def phen_data(files,fs):
    # Dataframe of the Phen data
    df = pd.read_csv(files)
    # ======== Read Phenomenological WF
    phen_t = df["Time"]
    phen_amp = df["H"]
    # ======== Numpy Array
    phen_t   = np.array(phen_t)
    phen_amp = np.array(phen_amp)
    # Code dissert that is in 4096 hz
    phen_t,phen_amp = resample(phen_t,phen_amp,fs)
    return phen_t,phen_amp

In [ ]:
def NoiseGen(asd,delta_f,tsamples,delta_t,seed):
    asd = np.array(asd)
    psd = pycbc.types.timeseries.FrequencySeries(asd,delta_f)**2
    nt = pycbc.noise.noise_from_psd(tsamples, delta_t, psd, seed=seed)

    return  nt

def PSDestimation(nt,delta_t,phen_amp,phen_t,fs,flow,factor):

    # Estimate noise PSD
    seg_len    = int(0.25 / delta_t)  # segments of 0.25s
    seg_stride = int(seg_len / 2)     # overlap of 50%
    PSDesti    = pycbc.psd.welch(nt, seg_len=seg_len, seg_stride=seg_stride )


    # Interpolate the PSD to match the size of ht
    Duration = len(phen_t)
    dfpsd = fs/Duration
    PSDesti = pycbc.psd.interpolate(PSDesti,1.0/Duration*fs)

    # Create a TimeSeries
    phen_amp = np.array(phen_amp)
    phen_amp_T = pycbc.types.timeseries.TimeSeries(phen_amp,delta_t)*factor

    #Calculate the SNR of the signal
    SNR = pycbc.filter.sigma(phen_amp_T, PSDesti, low_frequency_cutoff=flow)
    #print(f"The SNR of the signal: {SNR}")

    if SNR<18:
      for i in range(0,20):
        nfactor =  factor + .2e-21*i
        phen_amp_T = pycbc.types.timeseries.TimeSeries(phen_amp,delta_t)*nfactor
        SNR = pycbc.filter.sigma(phen_amp_T, PSDesti, low_frequency_cutoff=flow)
        if SNR<=24 and SNR>=18:
          print(f"The SNR of the signal: {SNR}, {nfactor:.2f}")
          break
    if SNR>24:
      #print("-")
      for i in range(0,20):
       #print("--")
       nfactor =  factor - .2e-21*i
       phen_amp_T = pycbc.types.timeseries.TimeSeries(phen_amp,delta_t)*nfactor
       SNR = pycbc.filter.sigma(phen_amp_T, PSDesti, low_frequency_cutoff=flow)

       if SNR<=24 and SNR>=18:
          print(f"The SNR of the signal: {SNR}, {nfactor:.2f}")
          break
    # for i in range(0,6):
    #     factor =  factor + .5e-21
    #     phen_amp_T = pycbc.types.timeseries.TimeSeries(phen_amp,delta_t)*factor
    #     SNR = pycbc.filter.sigma(phen_amp_T, PSDesti, low_frequency_cutoff=flow)
    #     #print(f"The SNR of the signal: {SNR}, {factor}")
    #     if SNR >=18 and SNR<=24:
    #       print(f"The SNR of the signal: {SNR}, {factor}")
    return phen_amp_T,PSDesti,SNR
    #     else:

In [ ]:
def Injector(nt,phen_amp,inf,sup,M,initial):
  '''
  inf = Inferior Limit of injection
  sup = Superior limit of injection
  M   = Number of injections
  '''

  nt_time = nt.sample_times
  #jitter value
  # M = 10
  nt = np.array(nt)
  L = np.zeros((M,len(nt)))
  jt = []
  for i in tqdm(range(0,M),leave=True,colour="yellow"):
    s = ran.randint(int(inf*fs),int(sup*fs))
    jt.append(s)
    L[i,:] = nt
    for j in tqdm(range(0,len(phen_amp)),colour="blue"):
      L[i,s+j] = nt[j+s] + phen_amp[j]
  if initial==True:
    pass
  else:
    pass
    # for i in tqdm(range(0,M)):
    #   L[i,:] = nt
    #   for j in tqdm(range(0,len(phen_amp))):
    #     L[i,v+j] = nt[j+v] + phen_amp[j]
  return L,jt


def Injector_setJitter(nt,phen_amp,s):
    """
    s = jitter
    """
    s = int(s)
    L = copy.copy(nt)
    phen_amp = np.array(phen_amp)

    L[s:s+len(phen_amp)] = L[s:s + len(phen_amp)] + phen_amp[0:len(phen_amp)]
    #print(L)
    # for i in tqdm(range(0,len(phen_amp)), leave=False):
    #   L[s+i] = nt[s+i] + phen_amp[i]
    return L

In [ ]:
def bandpass(white_strain,fmin,fmax):

    ts = pycbc.filter.highpass(white_strain, fmin,8)
    # plt.plot(ts.sample_times, ts, label='Highpassed')

    # Suppress the high frequencies
    ts = pycbc.filter.lowpass_fir(ts, fmax, 8)
    # plt.plot(ts.sample_times, ts, label='Highpassed + Lowpassed')

    # Estimate Bandpass PSD
    seg_len    = int(4 / delta_t)  # segments of 0.25s
    seg_stride = int(seg_len / 2)     # overlap of 50%
    psdts = pycbc.psd.welch(ts, seg_len=seg_len, seg_stride=seg_stride)


    return ts,psdts

In [ ]:
def Whiten_sig(L,cut,nw_samp,graphs,SNR,jitter): #10,0.5

  L= pycbc.types.timeseries.TimeSeries(L,delta_t)
  L_w,psd_lw = L.whiten(nw_samp,cut, return_psd=True)
  # Estimate noise PSD
  seg_len    = int(4 / delta_t)  # segments of 0.25s
  seg_stride = int(seg_len / 2)     # overlap of 50%
  psdL = pycbc.psd.welch(L, seg_len=seg_len, seg_stride=seg_stride )

  # Scale the whitenning
  fL = min(psdL**0.5)
  Lw1 = L_w * fL #min(psdL0**0.5) #5e-24
  # Estimate Whiten PSD
  psdLw1 = pycbc.psd.welch(Lw1, seg_len=seg_len, seg_stride=seg_stride)


  # #Bandpass Filter
  # ts = bandpass(Lw1,110,1600)
  # # Estimate Whiten-bandpass PSD
  # psdts = pycbc.psd.welch(ts, seg_len=seg_len, seg_stride=seg_stride)

  return Lw1, psdLw1

In [ ]:
# Generate N seconds of noise at 4096 Hz
N = 32
# Frequency Sample
fs = 4096
# Delta time of the noise window
delta_t = 1.0 / fs
# Number of samples in the noise window
tsamples = int(N / delta_t)
# Delta of the frequency of the frequency domain in the noise window
delta_f =1.0/16
flow = 1.0

Fm = 4096
r = int(Fm/16)
wnd= "blackman"
wnd= "hamming"

nfft=4*r
nperseg=r
noverlap=int(r-r/8)
spec_cmap='viridis'
#spec_cmap='coolwarm'

## GEN

In [ ]:
F = [0.1e-21,0.5e-21,1e-21,1.5e-21,2e-21,2.5e-21,3e-21]
SNR = np.zeros(len(F))
Signal = []
print(Signal)
for i in tqdm(range(0,2),colour="red",desc= f"Generación de señales SNR Fijo, "):

    nt = NoiseGen(L1_asd["ASD"],delta_f,tsamples,delta_t,ran.randint(0,120))
    # Noise PSD Estimation
    ph,psd_esti,SNR = PSDestimation(nt,delta_t,phen_amp,phen_t,fs,flow,1.0e-21)

    # if SNR<=18 or SNR>=24:

    # Injection in Noise by Phenom
    L,jitter = Injector(nt,ph,12,16,1,True)

    # Whitening
    Sw,Swpsd = Whiten_sig(L[0],10,0.5,False,SNR,jitter)

    # bandspass filter
    S,Spsd = bandpass(Sw,100,1600)

    #Calculate Spectrogram of the 32s
    fxx, txx, Sxx = sg.spectrogram(S, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )

    #
    n = ran.randint(int(0.1),int(4096-len(phen_amp)))
    print(f"Window injection at: {n/fs}")
    print("inint",jitter[0]- int(5*fs) - n)
    print("end:",jitter[0]- int(5*fs) - n + 4096)


    #Calculate Spectrogram of the Window of 1 sec
    win1_index_ini = jitter[0] - int(5*fs) - n
    win1_index_fin = win1_index_ini + 4096



    Swin = S[win1_index_ini:win1_index_fin]
    fxx1s, txx1s, Sxx1s = sg.spectrogram(Swin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )


    #Noise Win
    noise_win_end = win1_index_ini - fs
    noise_win_ini = noise_win_end  - fs

    Noisewin      = S[noise_win_ini:noise_win_end]

    # Calculate fxx
    fxxNL, txxNL, SxxNL = sg.spectrogram(Noisewin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )

    # # Create a figure and axes
    # fig, (ax1, ax2, ax3) = plt.subplots(3, 1,figsize=(12,6),layout="constrained")

    # # Plot the first spectrogram
    # #ax1.plot(S.sample_times,S)
    # ax1.loglog(psd_esti.sample_frequencies,psd_esti**0.5)
    # ax1.loglog(Swpsd.sample_frequencies,Swpsd**0.5)
    # ax1.loglog(Spsd.sample_frequencies,Spsd**0.5)


    # # # Plot the second spectrogram
    # ax2.pcolormesh( txx1s,fxx1s, Sxx1s,cmap=spec_cmap)
    # ax2.set_title(f"Window of 1s: L1 Injection Specgram, SNR:{SNR:.2f}") #injection at{jitter/fs -5 : .2f}
    # ax2.set_xlabel('Time [s]')
    # ax2.set_ylabel('Frequency [Hz]')
    # # ax2.set_xlim(Jitter[1][0]/fs - 5.5,Jitter[1][0]/fs - 4.5)
    # ax2.set_ylim(100,1500)

    # # Plot the third spectrogram
    # ax3.pcolormesh( txxNL, fxxNL, SxxNL,cmap=spec_cmap)
    # ax3.set_title(f"L1 Injection Specgram, SNR:{SNR:.2f}, injection at{jitter[0]/fs -5 : .2f}") #, injection at{Jitter[2][0]/fs -5 : .2f}
    # ax3.set_xlabel('Time [s]')
    # ax3.set_ylabel('Frequency [Hz]')
    # #ax3.set_xlim(jitter[0]/fs - 5.5,jitter[0]/fs - 4.5)
    # ax3.set_ylim(100,1500)

    # # Add colorbars to the spectrograms
    # # cbar1 = fig.colorbar(ax1.pcolormesh(txxGL0,fxxGL0,SxxGL0,cmap=spec_cmap), ax=ax1)
    # # # cbar1.set_label('Intensity')
    # cbar2 = fig.colorbar(ax2.pcolormesh( txx1s,fxx1s, Sxx1s,cmap=spec_cmap), ax=ax2)
    # # cbar2.set_label('Intensity')
    # cbar3 = fig.colorbar(ax3.pcolormesh( txxNL, fxxNL, SxxNL,cmap=spec_cmap), ax=ax3)

    # # # Adjust the spacing between subplots
    # # plt.constrain_layout()

    # # Display the figure
    # plt.show("off")

[]


Generación de señales SNR Fijo, :   0%|          | 0/2 [00:03<?, ?it/s]

The SNR of the signal: 10.541876288031453
The SNR of the signal: 10.541876288031453, 1e-21
The SNR of the signal: 15.81281443204718, 1.4999999999999998e-21
The SNR of the signal: 26.35469072007863, 2.4999999999999995e-21
The SNR of the signal: 42.167505152125806, 3.999999999999999e-21
The SNR of the signal: 63.25125772818872, 5.999999999999999e-21
The SNR of the signal: 89.60594844826737, 8.5e-21


TypeError: ignored

## Same Jitter

In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
import pandas as pd

DatL = pd.DataFrame()
DatH = pd.DataFrame()
DatV = pd.DataFrame()

DatN = pd.DataFrame()
cont = 0
for j in tqdm(range(0,100), colour="green",desc=f"Phenomenological Signal Development"):       #len(files))
  phen_t,phen_amp = phen_data(files[j],fs)
  for i in tqdm(range(0,10),colour="red",desc= f"Generación de señales SNR Fijo: (18-24)"):

      print("\t")
      # Create Noise
      # ====== L1
      ntL = NoiseGen(L1_asd["ASD"],delta_f,tsamples,delta_t,ran.randint(0,1e5))

      # ====== H1
      ntH = NoiseGen(H1_asd["ASD"],delta_f,tsamples,delta_t,ran.randint(0,1e5))

      # ====== V1
      ntV = NoiseGen(V1_asd["ASD"],delta_f,tsamples,delta_t,ran.randint(0,1e5))


      # Noise PSD Estimation
      # ====== L1
      phL,psd_estiL,SNRL = PSDestimation(ntL,delta_t,phen_amp,phen_t,fs,flow,1.5e-21)

    # ======  H1
      phH,psd_estiH,SNRH = PSDestimation(ntH,delta_t,phen_amp,phen_t,fs,flow,1.5e-21)

      # ====== V1
      phV,psd_estiV,SNRV = PSDestimation(ntV,delta_t,phen_amp,phen_t,fs,flow,1.5e-21)


      # Injection in Noise by Phenom
      # ====== L1
      LL,jitter = Injector(ntL,phL,12,16,1,True)
      # ====== H1
      LH = Injector_setJitter(ntH,phH,jitter[0])

      # ====== V1
      LV = Injector_setJitter(ntV,phV,jitter[0])


      # Whitening
      # ====== L1
      SLw,SLwpsd = Whiten_sig(LL[0],10,0.5,False,SNRL,jitter)

      # ====== H1
      SHw,SHwpsd = Whiten_sig(LH,10,0.5,False,SNRH,jitter)

      # ====== V1
      SVw,SVwpsd = Whiten_sig(LV,10,0.5,False,SNRV,jitter)

      # bandspass filter
      # ====== L1
      SL,SLpsd = bandpass(SLw,100,1800)
      # ====== H1
      SH,SHpsd = bandpass(SHw,100,1800)
      # ====== H1
      SV,SVpsd = bandpass(SVw,100,1800)

      #
      n = ran.randint(int(0.1),int(4096-len(phen_amp)))
      win1_index_ini = jitter[0] - int(5*fs) - n
      win1_index_fin = win1_index_ini + 4096

      #Calculate Spectrogram of the Window of 1 sec
      # # Calculate Spectrogram of the Window of 1 sec
      # win1_index_ini = jitter[0] - int(0.1*fs) - int(5*fs)
      # win1_index_fin = jitter[0] - int(0.1*fs) + 4096 - int(5*fs)

      # ====== L1
      SLwin = SL[win1_index_ini:win1_index_fin]
      #fxxL1s, txxL1s, SxxL1s = sg.spectrogram(SLwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )
      # ====== H1
      SHwin = SH[win1_index_ini:win1_index_fin]
      #fxxH1s, txxH1s, SxxH1s = sg.spectrogram(SHwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )
      # ====== V1
      SVwin = SV[win1_index_ini:win1_index_fin]
      #fxxV1s, txxV1s, SxxV1s = sg.spectrogram(SVwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )

      SLwin = np.array(SLwin)
      SHwin = np.array(SHwin)
      SVwin = np.array(SVwin)
      # Data Save
      # DatL[f"N{i}"] = SLwin
      # DatH[f"N{i}"] = SHwin
      # DatV[f"N{i}"] = SVwin

      DatL[f"N{j}{i}"] = SLwin
      DatH[f"N{j}{i}"] = SHwin
      DatV[f"N{j}{i}"] = SVwin


      # Noise Win
      noise_win_end = win1_index_ini - fs
      noise_win_ini = noise_win_end  - fs

      NoiseLwin      = SL[noise_win_ini:noise_win_end]
      NoiseHwin      = SH[noise_win_ini:noise_win_end]
      NoiseVwin      = SV[noise_win_ini:noise_win_end]

      NoiseLwin = np.array(NoiseLwin)
      NoiseHwin = np.array(NoiseHwin)
      NoiseVwin = np.array(NoiseVwin)

      # Save Noise Data
      DatN[f"NL{j}{i}"] = NoiseLwin
      DatN[f"NH{j}{i}"] = NoiseHwin
      DatN[f"NW{j}{i}"] = NoiseVwin



Generación de señales SNR Fijo: (18-24):   0%|          | 0/10 [00:00<?, ?it/s]

	
The SNR of the signal: 18.986829691267566, 0.00
The SNR of the signal: 18.879157248499123, 0.00
The SNR of the signal: 18.929671491798953, 0.00




  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 69.32it/s]

Generación de señales SNR Fijo: (18-24):  10%|█         | 1/10 [00:22<03:18, 22.00s/it]

	
The SNR of the signal: 18.969741339244138, 0.00
The SNR of the signal: 18.938191207901767, 0.00
The SNR of the signal: 18.811169607267622, 0.00




  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 37.54it/s]

Generación de señales SNR Fijo: (18-24):  20%|██        | 2/10 [00:44<03:00, 22.56s/it]

...



  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 62.99it/s]

Generación de señales SNR Fijo: (18-24):  60%|██████    | 6/10 [01:42<01:07, 16.90s/it]



  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 53.62it/s]

Generación de señales SNR Fijo: (18-24):  70%|███████   | 7/10 [01:58<00:50, 16.72s/it]



  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 65.79it/s]

Generación de señales SNR Fijo: (18-24):  80%|████████  | 8/10 [02:14<00:33, 16.50s/it]



  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 52.40it/s]

Generación de señales SNR Fijo: (18-24):  90%|█████████ | 9/10 [02:32<00:16, 16.82s/it]



  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 64.56it/s]

Phenomenological Signal Development: 100%|██████████| 100/100 [4:39:01<00:00, 167.41s/it]


In [ ]:
DatN.to_csv("DataNoise3000.csv")

In [ ]:
DatH.to_csv("DataH1000.csv")
DatL.to_csv("DataL1000.csv")
DatV.to_csv("DataV1000.csv")

In [ ]:
import PIL
from PIL import Image

Fm = 4096
r = int(Fm/16)
wnd= "hamming"

nfft=4*r
nperseg=r
noverlap=int(r-r/8)
spec_cmap='viridis'
#spec_cmap='coolwarm'

# data_stor = []

# for i in tqdm(DatN.keys()):
#     fxx,txx,Sxx = sg.spectrogram(DatN[f"{i}"],Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude')

#     fig = plt.figure()
#     fig.patch.set_visible(False)
#     plt.axis("off")
#     plt.pcolormesh(txx,fxx,Sxx,vmin=0e-23,vmax=6e-23)
#     plt.ylim(0,1800)
#     plt.tight_layout(pad=0)
#     plt.close()
#     fig.canvas.draw()

#     data1 = np.frombuffer(fig.canvas.tostring_rgb(),dtype=np.uint8)
#     datas = data1.reshape(fig.canvas.get_width_height()[::-1] + (3,))

#     img = Image.fromarray(datas)
#     img.show()
#     img_L = img.convert("L")

#     img_L.show()
#     img_res = img_L.resize((10,10))
#     img_resnp = np.array(img_res)
#     data_stor.append(img_resnp)

def SpecExtrac(Dataset):
    """

    R = Number of Pixels
    """

    fxx,txx,Sxx = sg.spectrogram(Dataset,Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude')
    fig = plt.figure()
    fig.patch.set_visible(False)
    plt.axis("off")
    plt.pcolormesh(txx,fxx,Sxx)
    plt.ylim(100,1800)
    plt.tight_layout(pad=0)
    plt.close()
    fig.canvas.draw()

    data1 = np.frombuffer(fig.canvas.tostring_rgb(),dtype=np.uint8)
    datas = data1.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    img = Image.fromarray(datas)
    #img.show()
    img_L = img.convert("L")
    #img_L.show()
    img_res = img_L.resize((28,28))
    #img_res.show()
    img_resnp = np.array(img_res)
    img_resFlat = img_resnp.flatten()
    #data_stor.append(img_resnp)
    return img_resFlat


In [ ]:
DatL.columns[0]

'N00'

In [ ]:
SpecL = pd.DataFrame()
SpecH = pd.DataFrame()
SpecV = pd.DataFrame()

SpecN = pd.DataFrame()
for i in tqdm(range(0,1000),desc=f"{i}"):
    im = SpecExtrac(DatL[DatL.columns[i]])
    SpecL[f"{i}"] = im

11: 100%|██████████| 1000/1000 [02:28<00:00,  6.71it/s]


In [ ]:
SpecH.to_csv("SpecH.csv")

In [ ]:
import gc
#del SpecV
gc.collect()

8

In [ ]:
# for i in tqdm(range(0,1000),desc=f"{i}"):
#     im = SpecExtrac(DatH[DatH.columns[i]])
#     SpecH[f"{i}"] = im

# for i in tqdm(range(0,1000),desc=f"{i}"):
#     im = SpecExtrac(DatV[DatV.columns[i]])
#     SpecV[f"{i}"] = im
SpecN = pd.DataFrame()
for i in tqdm(range(0,3000),desc=f"{i}"):
    im = SpecExtrac(DatN[DatN.columns[i]])
    SpecN[f"{i}"] = im
SpecN.to_csv("SpecN.csv")


999:  47%|████▋     | 1419/3000 [03:29<03:53,  6.76it/s]


KeyboardInterrupt: ignored

In [ ]:
print(R)

     140  141  142  143  144  145  146  147  148  149
0     63   50   48   57   68   63   50   66   49   54
1     60   54   58   56   65   53   49   60   55   48
2     52   48   57   53   69   62   50   49   59   44
3     63   51   62   51   66   57   53   52   52   42
4     63   52   59   49   66   60   51   51   47   45
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
779   60   68   66   74   73   57   66   67   60   62
780   64   73   54   77   81   66   69   64   58   56
781   69   56   57   75   73   70   74   71   60   59
782   65   72   68   77   73   65   65   68   62   55
783   78   80   64   68   78   88   71   61   66   60

[784 rows x 10 columns]


In [ ]:
      # Calculate Spectrogram
      # # ====== L1
      # fxxNL, txxNL, SxxNL = sg.spectrogram(NoiseLwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )
      # # ====== H1
      # fxxNH, txxNH, SxxNH = sg.spectrogram(NoiseHwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )
      # # ====== V1
      # fxxNV, txxNV, SxxNV = sg.spectrogram(NoiseVwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )



    # # Create a figure and axes
    # fig, (ax1, ax2, ax3) = plt.subplots(3, 1,figsize=(12,6),layout="constrained")

    # # Plot the first spectrogram
    # ax1.pcolormesh(txxL1s,fxxL1s, SxxL1s,cmap=spec_cmap)
    # ax1.set_title(f"Window of 1s: L1 Injection Specgram, SNR:{SNRL:.2f}") #injection at{jitter/fs -5 : .2f}
    # ax1.set_xlabel('Time [s]')
    # ax1.set_ylabel('Frequency [Hz]')
    # # ax2.set_xlim(Jitter[1][0]/fs - 5.5,Jitter[1][0]/fs - 4.5)
    # ax1.set_ylim(100,1500)


    # # # Plot the second spectrogram
    # ax2.pcolormesh(txxH1s,fxxH1s, SxxH1s,cmap=spec_cmap)
    # ax2.set_title(f"Window of 1s: H1 Injection Specgram, SNR:{SNRH:.2f}") #injection at{jitter/fs -5 : .2f}
    # ax2.set_xlabel('Time [s]')
    # ax2.set_ylabel('Frequency [Hz]')
    # # ax2.set_xlim(Jitter[1][0]/fs - 5.5,Jitter[1][0]/fs - 4.5)
    # ax2.set_ylim(100,1500)

    # # Plot the third spectrogram
    # ax3.pcolormesh( txxV1s, fxxV1s, SxxV1s,cmap=spec_cmap)
    # ax3.set_title(f"Window of 1s: V1 Injection Specgram, SNR:{SNRV:.2f}") #, injection at{Jitter[2][0]/fs -5 : .2f}
    # ax3.set_xlabel('Time [s]')
    # ax3.set_ylabel('Frequency [Hz]')
    # #ax3.set_xlim(jitter[0]/fs - 5.5,jitter[0]/fs - 4.5)
    # ax3.set_ylim(100,1500)

    # # Add colorbars to the spectrograms
    # cbar1 = fig.colorbar(ax1.pcolormesh(txxL1s,fxxL1s, SxxL1s,cmap=spec_cmap), ax=ax1)
    #  # cbar1.set_label('Intensity')
    # cbar2 = fig.colorbar(ax2.pcolormesh(txxH1s,fxxH1s, SxxH1s,cmap=spec_cmap), ax=ax2)
    # # cbar2.set_label('Intensity')
    # cbar3 = fig.colorbar(ax3.pcolormesh(txxV1s,fxxV1s, SxxV1s,cmap=spec_cmap), ax=ax3)

    # # # Adjust the spacing between subplots
    # # plt.constrain_layout()

    # # Display the figure
    # plt.show()


    # # Create a figure and axes - NOISE
    # fig, (ax1, ax2, ax3) = plt.subplots(3, 1,figsize=(12,6),layout="constrained")

    # # Plot the first spectrogram
    # ax1.pcolormesh(txxNL,fxxNL, SxxNL,cmap=spec_cmap)
    # ax1.set_title(f"Window of 1s: L1 Noise Specgram, SNR:{SNRL:.2f}") #injection at{jitter/fs -5 : .2f}
    # ax1.set_xlabel('Time [s]')
    # ax1.set_ylabel('Frequency [Hz]')
    # # ax2.set_xlim(Jitter[1][0]/fs - 5.5,Jitter[1][0]/fs - 4.5)
    # ax1.set_ylim(100,1500)


    # # # Plot the second spectrogram
    # ax2.pcolormesh(txxNH,fxxNH, SxxNH,cmap=spec_cmap)
    # ax2.set_title(f"Window of 1s: H1 Noise Specgram, SNR:{SNRH:.2f}") #injection at{jitter/fs -5 : .2f}
    # ax2.set_xlabel('Time [s]')
    # ax2.set_ylabel('Frequency [Hz]')
    # # ax2.set_xlim(Jitter[1][0]/fs - 5.5,Jitter[1][0]/fs - 4.5)
    # ax2.set_ylim(100,1500)

    # # Plot the third spectrogram
    # ax3.pcolormesh( txxNV, fxxNV, SxxNV,cmap=spec_cmap)
    # ax3.set_title(f"Window of 1s: H1 Noise Specgram, SNR:{SNRV:.2f}") #, injection at{Jitter[2][0]/fs -5 : .2f}
    # ax3.set_xlabel('Time [s]')
    # ax3.set_ylabel('Frequency [Hz]')
    # #ax3.set_xlim(jitter[0]/fs - 5.5,jitter[0]/fs - 4.5)
    # ax3.set_ylim(100,1500)

    # # Add colorbars to the spectrograms
    # cbar1 = fig.colorbar(ax1.pcolormesh(txxNL,fxxNL, SxxNL,cmap=spec_cmap), ax=ax1)
    #  # cbar1.set_label('Intensity')
    # cbar2 = fig.colorbar(ax2.pcolormesh(txxNH,fxxNH, SxxNH,cmap=spec_cmap), ax=ax2)
    # # cbar2.set_label('Intensity')
    # cbar3 = fig.colorbar(ax3.pcolormesh(txxNV, fxxNV, SxxNV,cmap=spec_cmap), ax=ax3)

    # # # Adjust the spacing between subplots
    # # plt.constrain_layout()

    # # Display the figure
    # plt.show()

In [ ]:
DatH

,N0,N00,N1,N01,N2,N02,N3,N03,N4,N04,...,N20,N21,N22,N23,N24,N25,N26,N27,N28,N29
0,-9.104894e-23,-1.946960e-23,6.200001e-23,-6.621689e-24,6.359287e-23,-1.251453e-22,6.357579e-23,6.835274e-23,-4.092391e-23,-8.724413e-23,...,-9.104894e-23,6.200001e-23,6.359287e-23,6.357579e-23,-4.092391e-23,6.457162e-24,1.023142e-22,1.533237e-22,1.466521e-23,1.556688e-22
1,-1.465953e-22,-2.361346e-23,-1.389773e-22,8.951408e-23,-3.852097e-23,1.019713e-23,-1.356397e-22,-6.829888e-23,9.277555e-23,-9.827220e-24,...,-1.465953e-22,-1.389773e-22,-3.852097e-23,-1.356397e-22,9.277555e-23,-1.576201e-22,-1.167489e-22,1.210638e-22,-4.404337e-23,1.100807e-23
2,1.699313e-23,2.593356e-23,7.956306e-23,6.301939e-23,-1.402044e-22,-6.328951e-23,-2.217748e-24,1.898533e-23,7.804715e-23,5.621273e-23,...,1.699313e-23,7.956306e-23,-1.402044e-22,-2.217748e-24,7.804715e-23,-9.896548e-23,-7.416879e-23,-1.037979e-22,1.446407e-23,1.118854e-22
3,2.444933e-23,9.371051e-23,1.146179e-22,-7.943230e-23,-7.591833e-23,-1.218998e-23,4.691833e-23,6.808126e-23,1.546844e-22,-8.580438e-23,...,2.444933e-23,1.146179e-22,-7.591833e-23,4.691833e-23,1.546844e-22,-3.646979e-23,8.972996e-23,4.236480e-23,-6.538021e-23,3.399822e-23
4,1.936540e-23,-6.242193e-23,-6.821598e-23,9.099698e-23,-2.090781e-22,1.726500e-22,-9.454196e-23,-1.943825e-22,1.223564e-22,3.683849e-23,...,1.936540e-23,-6.821598e-23,-2.090781e-22,-9.454196e-23,1.223564e-22,-2.129563e-22,-8.906888e-23,9.839333e-23,7.089022e-23,-2.292944e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4.281326e-23,-6.706745e-23,9.985943e-23,-5.182833e-23,-4.009210e-23,-3.489546e-23,1.902149e-22,8.230152e-23,2.910407e-24,8.076861e-23,...,4.281326e-23,9.985943e-23,-4.009210e-23,1.902149e-22,2.910407e-24,2.031629e-22,7.795708e-23,8.028010e-23,-4.339189e-23,-3.613106e-23
4092,-2.097957e-23,8.578258e-23,5.437604e-23,1.328438e-22,-5.705541e-25,-5.184866e-23,2.773854e-22,-7.627185e-23,-1.051415e-22,6.379414e-23,...,-2.097957e-23,5.437604e-23,-5.705541e-25,2.773854e-22,-1.051415e-22,-9.585237e-23,1.544644e-22,-2.085476e-23,1.215870e-22,-4.663049e-23
4093,-1.805614e-22,-4.446426e-23,4.204353e-23,3.031588e-23,1.117813e-22,-1.672411e-22,7.587583e-23,-1.452481e-22,-2.024084e-22,-1.191772e-22,...,-1.805614e-22,4.204353e-23,1.117813e-22,7.587583e-23,-2.024084e-22,-1.799660e-22,-1.677437e-22,-2.783617e-23,1.171823e-22,1.043630e-22
4094,-6.045027e-23,1.657489e-22,-4.529744e-23,-1.953053e-22,-2.151802e-24,-2.260683e-23,-1.061544e-22,-1.089041e-22,-2.980906e-23,-1.298767e-22,...,-6.045027e-23,-4.529744e-23,-2.151802e-24,-1.061544e-22,-2.980906e-23,-4.790170e-24,-4.204496e-23,1.538582e-22,1.747533e-23,1.075134e-22


In [ ]:
import pandas as pd
# for j in tqdm(range(0,len(files))):

  # phen_t,phen_amp = phen_data(files,fs)

DatL = pd.DataFrame()
DatH = pd.DataFrame()
DatV = pd.DataFrame()

DatN = pd.DataFrame()


for i in tqdm(range(0,10),colour="red",desc= f"Generación de señales SNR Fijo, "):
    print("\t")
    # Create Noise
    # ====== L1
    ntL = NoiseGen(L1_asd["ASD"],delta_f,tsamples,delta_t,ran.randint(0,1e5))

    # ====== H1
    ntH = NoiseGen(H1_asd["ASD"],delta_f,tsamples,delta_t,ran.randint(0,1e5))

    # ====== V1
    ntV = NoiseGen(V1_asd["ASD"],delta_f,tsamples,delta_t,ran.randint(0,1e5))


    # Noise PSD Estimation
    # ====== L1
    phL,psd_estiL,SNRL = PSDestimation(ntL,delta_t,phen_amp,phen_t,fs,flow,1.5e-21)

   # ======  H1
    phH,psd_estiH,SNRH = PSDestimation(ntH,delta_t,phen_amp,phen_t,fs,flow,1.5e-21)

    # ====== V1
    phV,psd_estiV,SNRV = PSDestimation(ntV,delta_t,phen_amp,phen_t,fs,flow,1.5e-21)


    # Injection in Noise by Phenom
    # ====== L1
    LL,jitter = Injector(ntL,phL,12,16,1,True)
    # ====== H1
    LH = Injector_setJitter(ntH,phH,jitter[0])

    # ====== V1
    LV = Injector_setJitter(ntV,phV,jitter[0])



    # Whitening
    # ====== L1
    SLw,SLwpsd = Whiten_sig(LL[0],10,0.5,False,SNRL,jitter)

    # ====== H1
    SHw,SHwpsd = Whiten_sig(LH,10,0.5,False,SNRH,jitter)

    # ====== V1
    SVw,SVwpsd = Whiten_sig(LV,10,0.5,False,SNRV,jitter)

    # bandspass filter
    # ====== L1
    SL,SLpsd = bandpass(SLw,100,1800)
    # ====== H1
    SH,SHpsd = bandpass(SHw,100,1800)
    # ====== H1
    SV,SVpsd = bandpass(SVw,100,1800)

    # Calculate Spectrogram of the Window of 1 sec
    win1_index_ini = jitter[0] - int(0.1*fs) - int(5*fs)
    win1_index_fin = jitter[0] - int(0.1*fs) + 4096 - int(5*fs)

    # ====== L1
    SLwin = SL[win1_index_ini:win1_index_fin]
    fxxL1s, txxL1s, SxxL1s = sg.spectrogram(SLwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )
    # ====== H1
    SHwin = SH[win1_index_ini:win1_index_fin]
    fxxH1s, txxH1s, SxxH1s = sg.spectrogram(SHwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )
    # ====== V1
    SVwin = SV[win1_index_ini:win1_index_fin]
    fxxV1s, txxV1s, SxxV1s = sg.spectrogram(SVwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )

    SLwin = np.array(SLwin)
    SHwin = np.array(SHwin)
    SVwin = np.array(SVwin)
    # Data Save
    DatL[f"N{i}"] = SLwin
    DatH[f"N{i}"] = SHwin
    DatV[f"N{i}"] = SVwin



    # Noise Win
    noise_win_end = win1_index_ini - fs
    noise_win_ini = noise_win_end  - fs

    NoiseLwin      = SL[noise_win_ini:noise_win_end]
    NoiseHwin      = SH[noise_win_ini:noise_win_end]
    NoiseVwin      = SV[noise_win_ini:noise_win_end]

    NoiseLwin = np.array(NoiseLwin)
    NoiseHwin = np.array(NoiseHwin)
    NoiseVwin = np.array(NoiseVwin)

    # Save Noise Data
    DatN[f"NL{i}"] = NoiseLwin
    DatN[f"NH{i}"] = NoiseHwin
    DatN[f"NW{i}"] = NoiseVwin

    # Calculate Spectrogram
    # ====== L1
    fxxNL, txxNL, SxxNL = sg.spectrogram(NoiseLwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )
    # ====== H1
    fxxNH, txxNH, SxxNH = sg.spectrogram(NoiseHwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )
    # ====== V1
    fxxNV, txxNV, SxxNV = sg.spectrogram(NoiseVwin, Fm, window=wnd, nfft=nfft,nperseg=nperseg ,noverlap=noverlap, mode='magnitude' )



    # # Create a figure and axes
    # fig, (ax1, ax2, ax3) = plt.subplots(3, 1,figsize=(12,6),layout="constrained")

    # # Plot the first spectrogram
    # ax1.pcolormesh(txxL1s,fxxL1s, SxxL1s,cmap=spec_cmap)
    # ax1.set_title(f"Window of 1s: L1 Injection Specgram, SNR:{SNRL:.2f}") #injection at{jitter/fs -5 : .2f}
    # ax1.set_xlabel('Time [s]')
    # ax1.set_ylabel('Frequency [Hz]')
    # # ax2.set_xlim(Jitter[1][0]/fs - 5.5,Jitter[1][0]/fs - 4.5)
    # ax1.set_ylim(100,1500)


    # # # Plot the second spectrogram
    # ax2.pcolormesh(txxH1s,fxxH1s, SxxH1s,cmap=spec_cmap)
    # ax2.set_title(f"Window of 1s: H1 Injection Specgram, SNR:{SNRH:.2f}") #injection at{jitter/fs -5 : .2f}
    # ax2.set_xlabel('Time [s]')
    # ax2.set_ylabel('Frequency [Hz]')
    # # ax2.set_xlim(Jitter[1][0]/fs - 5.5,Jitter[1][0]/fs - 4.5)
    # ax2.set_ylim(100,1500)

    # # Plot the third spectrogram
    # ax3.pcolormesh( txxV1s, fxxV1s, SxxV1s,cmap=spec_cmap)
    # ax3.set_title(f"Window of 1s: V1 Injection Specgram, SNR:{SNRV:.2f}") #, injection at{Jitter[2][0]/fs -5 : .2f}
    # ax3.set_xlabel('Time [s]')
    # ax3.set_ylabel('Frequency [Hz]')
    # #ax3.set_xlim(jitter[0]/fs - 5.5,jitter[0]/fs - 4.5)
    # ax3.set_ylim(100,1500)

    # # Add colorbars to the spectrograms
    # cbar1 = fig.colorbar(ax1.pcolormesh(txxL1s,fxxL1s, SxxL1s,cmap=spec_cmap), ax=ax1)
    #  # cbar1.set_label('Intensity')
    # cbar2 = fig.colorbar(ax2.pcolormesh(txxH1s,fxxH1s, SxxH1s,cmap=spec_cmap), ax=ax2)
    # # cbar2.set_label('Intensity')
    # cbar3 = fig.colorbar(ax3.pcolormesh(txxV1s,fxxV1s, SxxV1s,cmap=spec_cmap), ax=ax3)

    # # # Adjust the spacing between subplots
    # # plt.constrain_layout()

    # # Display the figure
    # plt.show()


    # # Create a figure and axes - NOISE
    # fig, (ax1, ax2, ax3) = plt.subplots(3, 1,figsize=(12,6),layout="constrained")

    # # Plot the first spectrogram
    # ax1.pcolormesh(txxNL,fxxNL, SxxNL,cmap=spec_cmap)
    # ax1.set_title(f"Window of 1s: L1 Noise Specgram, SNR:{SNRL:.2f}") #injection at{jitter/fs -5 : .2f}
    # ax1.set_xlabel('Time [s]')
    # ax1.set_ylabel('Frequency [Hz]')
    # # ax2.set_xlim(Jitter[1][0]/fs - 5.5,Jitter[1][0]/fs - 4.5)
    # ax1.set_ylim(100,1500)


    # # # Plot the second spectrogram
    # ax2.pcolormesh(txxNH,fxxNH, SxxNH,cmap=spec_cmap)
    # ax2.set_title(f"Window of 1s: H1 Noise Specgram, SNR:{SNRH:.2f}") #injection at{jitter/fs -5 : .2f}
    # ax2.set_xlabel('Time [s]')
    # ax2.set_ylabel('Frequency [Hz]')
    # # ax2.set_xlim(Jitter[1][0]/fs - 5.5,Jitter[1][0]/fs - 4.5)
    # ax2.set_ylim(100,1500)

    # # Plot the third spectrogram
    # ax3.pcolormesh( txxNV, fxxNV, SxxNV,cmap=spec_cmap)
    # ax3.set_title(f"Window of 1s: H1 Noise Specgram, SNR:{SNRV:.2f}") #, injection at{Jitter[2][0]/fs -5 : .2f}
    # ax3.set_xlabel('Time [s]')
    # ax3.set_ylabel('Frequency [Hz]')
    # #ax3.set_xlim(jitter[0]/fs - 5.5,jitter[0]/fs - 4.5)
    # ax3.set_ylim(100,1500)

    # # Add colorbars to the spectrograms
    # cbar1 = fig.colorbar(ax1.pcolormesh(txxNL,fxxNL, SxxNL,cmap=spec_cmap), ax=ax1)
    #  # cbar1.set_label('Intensity')
    # cbar2 = fig.colorbar(ax2.pcolormesh(txxNH,fxxNH, SxxNH,cmap=spec_cmap), ax=ax2)
    # # cbar2.set_label('Intensity')
    # cbar3 = fig.colorbar(ax3.pcolormesh(txxNV, fxxNV, SxxNV,cmap=spec_cmap), ax=ax3)

    # # # Adjust the spacing between subplots
    # # plt.constrain_layout()

    # # Display the figure
    # plt.show()

In [ ]:
DatH

,N0,N1,N2,N3,N4,N5,N6,N7,N8,N9
0,-1.124935e-22,-1.127021e-22,9.338475e-23,-1.082205e-22,-3.905622e-23,3.386287e-23,-1.389675e-22,-7.924020e-23,-7.742868e-23,-1.826900e-22
1,-5.655767e-23,-1.183961e-22,4.304110e-23,-4.488279e-23,9.679605e-23,-9.172831e-23,-3.276593e-23,-9.796995e-23,5.723700e-23,-1.146448e-22
2,1.918073e-22,3.326912e-23,-7.860164e-23,-3.336670e-24,-1.036702e-23,-7.129698e-23,-3.640259e-23,1.026671e-22,8.191776e-23,1.289900e-22
3,1.761669e-22,-3.299473e-23,-5.064833e-23,4.349902e-23,-2.622809e-23,1.037805e-22,7.046713e-23,7.432705e-23,-1.134045e-22,-1.519001e-22
4,-9.942220e-23,3.029224e-23,6.218285e-23,-3.767347e-23,7.635807e-23,2.758442e-23,8.408846e-23,-1.170158e-22,-2.038752e-23,8.838460e-23
...,...,...,...,...,...,...,...,...,...,...
4091,3.452825e-23,8.443949e-23,-4.231723e-23,-3.784274e-23,-3.403693e-23,4.225499e-23,1.835137e-22,-1.098051e-22,-6.951765e-23,1.007645e-22
4092,-5.336033e-23,5.734263e-23,1.923054e-22,2.430966e-23,-4.040872e-24,-1.245199e-22,-8.786035e-23,1.084043e-22,2.498879e-22,9.245491e-23
4093,-1.265422e-22,-1.185407e-23,1.008730e-23,1.013716e-22,1.735756e-23,1.695542e-23,-4.242754e-23,1.594302e-22,-1.569471e-23,6.382388e-23
4094,-4.230846e-23,1.364267e-22,-1.314071e-22,9.067134e-23,2.995575e-23,-2.277178e-23,1.886822e-22,3.151225e-23,-1.172232e-23,2.673797e-24
